In [7]:
!pip install pyDOE
!pip install cma

In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.36 ,random_state=0 , l1_ratio=0.87, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_2000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_15_400Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [11]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [12]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,101):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 100, [5] * 100 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 100000)

n_obs , dim =  30, 100
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,300])
for i in range(G[1].shape[0]):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(8_w,17)-aCMA-ES (mu_w=5.1,w_1=31%) in dimension 100 (seed=957165, Tue Apr 14 12:11:43 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     17 9.981686565635564e+03 1.0e+00 2.40e+00  2e+00  2e+00 0:00.0
    2     34 9.156963031907715e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 6.845791222996202e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [12192.17066092] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [9394.65583611] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   87   1479 -1.184214591774707e+04 1.3e+00 8.44e-01  8e-01  9e-01 0:03.1
  100   1700 -1.320735153123567e+04 1.3e+00 6.71e-01  7e-01  7e-01 0:03.6
  200   3400 -1.900679411893984e+04 1.3e+00 1.17e-01  1e-01  1e-01 0:07.1
  300   5100 -1.915991660279688e+04 1.3e+00 1.86e-02  2e-02  2e-02 0:10.6
  400   6800 -1.916359928178995e+04 1.3e+00 3.05e-03  3e-03  3e-03 0:14.1
  500   8500 -1.916371277590298e+04 1.3e+00 5.17e-04  5e-04  5e-04 0:17.6
  600  10200 -1.916371618279171e+04 1.3e+00 8.59e-05  8e-05  8e-05 0:21.1
  700  11900 -1.916371625560287e+04 1.4e+00 1.32e-05  1e-05  1e-05 0:24.6
  800  13600 -1.916371625798516e+04 1.4e+00 2.10e-06  2e-06  2e-06 0:28.2
  900  15300 -1.916371625804243e+04 1.4e+00 3.40e-07  3e-07  3e-07 0:31.7
 1000  17000 -1.916371625804349e+04 1.4e+00 5.92e-08  5e-08  6e-08 0:35.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.916371625804352e+04 1.4e+00 2.76e-08  2e-08  3e-08 0:38.7
 1200  20400 -1.916371625804352e+04 1.5e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10895.81357622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

    2     34 9.833546343941414e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 8.589541644569737e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1
   78   1326 -2.660167514560047e+03 1.3e+00 1.18e+00  1e+00  1e+00 0:03.1
  100   1700 -3.365956245296012e+03 1.3e+00 8.72e-01  9e-01  9e-01 0:04.0
  200   3400 -7.268087630987761e+03 1.4e+00 2.61e-01  2e-01  3e-01 0:08.0
  300   5100 -7.866021295883979e+03 1.5e+00 9.40e-02  9e-02  1e-01 0:12.0
  400   6800 -7.984937556680347e+03 1.8e+00 4.36e-02  4e-02  5e-02 0:16.1
  500   8500 -8.012799934316311e+03 2.2e+00 2.57e-02  2e-02  3e-02 0:20.1
  600  10200 -8.033365390987949e+03 3.1e+00 2.94e-02  3e-02  4e-02 0:24.1
  700  11900 -8.060589070394178e+03 4.2e+00 2.76e-02  2e-02  5e-02 0:28.1
  800  13600 -8.088890805272146e+03 4.5e+00 3.32e-02  3e-02  8e-02 0:32.2
  900  15300 -8.117940326039658e+03 4.8e+00 2.39e-02  2e-02  6e-02 0:36.2
 1000  17000 -8.126063552952194e+03 4.8e+00 1.65e-02  1e-02  4e-02 0:40.2
Iterat #Fevals   function value  axis ra

...

   87   1479 -8.728520811479879e+03 1.2e+00 1.05e+00  1e+00  1e+00 0:03.1
  100   1700 -1.073808556052913e+04 1.2e+00 8.64e-01  8e-01  9e-01 0:03.6
  200   3400 -1.878223142883482e+04 1.2e+00 1.61e-01  2e-01  2e-01 0:07.1
  300   5100 -1.915318443280701e+04 1.3e+00 2.65e-02  3e-02  3e-02 0:10.6
  400   6800 -1.916342319202497e+04 1.3e+00 4.81e-03  5e-03  5e-03 0:14.1
  500   8500 -1.916370874203004e+04 1.3e+00 8.03e-04  8e-04  8e-04 0:17.6
  600  10200 -1.916371602629194e+04 1.3e+00 1.32e-04  1e-04  1e-04 0:21.1
  700  11900 -1.916371625142871e+04 1.3e+00 2.48e-05  2e-05  2e-05 0:24.6
  800  13600 -1.916371625781471e+04 1.3e+00 4.50e-06  4e-06  4e-06 0:28.1
  900  15300 -1.916371625803821e+04 1.3e+00 7.01e-07  6e-07  7e-07 0:31.6
 1000  17000 -1.916371625804340e+04 1.4e+00 1.25e-07  1e-07  1e-07 0:35.1
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -1.916371625804351e+04 1.4e+00 3.39e-08  3e-08  3e-08 0:38.6
 1200  20400 -1.916371625804352e+04 1.4e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [10895.81357622] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=10895.813576, sigma=2.40e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  '

    2     34 6.906491059241682e+03 1.0e+00 2.33e+00  2e+00  2e+00 0:00.1
    3     51 6.475127264722549e+03 1.0e+00 2.27e+00  2e+00  2e+00 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [8684.42201159] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   77   1309 -1.557897851562403e+03 1.2e+00 1.17e+00  1e+00  1e+00 0:03.1
  100   1700 -2.691242038647880e+03 1.3e+00 8.05e-01  8e-01  8e-01 0:04.1
  200   3400 -5.582929864691700e+03 1.3e+00 2.75e-01  3e-01  3e-01 0:08.2
  300   5100 -6.518555091236972e+03 1.5e+00 1.29e-01  1e-01  1e-01 0:12.2
  400   6800 -7.279193174803112e+03 2.2e+00 1.65e-01  1e-01  2e-01 0:16.3
  500   8500 -7.974080316152963e+03 2.3e+00 1.30e-01  1e-01  2e-01 0:20.4
  600  10200 -8.362239259555492e+03 2.4e+00 1.01e-01  9e-02  2e-01 0:24.5
  700  11900 -8.640071220502847e+03 2.7e+00 1.01e-01  8e-02  2e-01 0:28.5
  800  13600 -8.802916631201482e+03 3.0e+00 6.17e-02  5e-02  9e-02 0:32.6
  900  15300 -8.829348457065920e+03 3.0e+00 2.34e-02  2e-02  4e-02 0:36.8
 1000  17000 -8.836632259986760e+03 3.0e+00 1.37e-02  1e-02  2e-02 0:40.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  18700 -8.840499959804056e+03 3.0e+00 1.01e-02  8e-03  2e-02 0:45.0
 1200  20400 -8.842642451381216e+03 3.5e

In [13]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F15_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:100])
    SVM_Fun [i] = F15(X_Values.iloc[i,100:200])
    ELN_Fun [i] = F15(X_Values.iloc[i,200:300])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2034.46518598]), count=array([1])) 4.64643220208136e-05
SVM
ModeResult(mode=array([6188.07060932]), count=array([1])) 2982.1538184305555
ELN
ModeResult(mode=array([8985.37123195]), count=array([1])) 2379.6548149197
